# TileDB VCF Example for 1000 Genomes Project

This document is an example walkthrough of [TileDB-VCF](https://github.com/TileDB-Inc/TileDB-VCF.git) using Phase 3 of the [1000 Genomes Project](https://www.internationalgenome.org/) data. The goal is to highlight storing these samples in TileDB-VCF along with querying in python and exporting back to VCF and TSV.

This expands upon the with specific use case. Please see the [official documentation](https://docs.tiledb.com/genomics/) for details on installing TileDB-VCF and more advanced usage.

## Preprocessing the Raw 1KG pVCF File

The original VCF file was downloaded from the AWS Open Data Registry: <a href="https://registry.opendata.aws/1000-genomes" class="uri">https://registry.opendata.aws/1000-genomes</a>.

In [2]:
!aws s3 sync \
    --exclude "*" --include "ALL.chr1.*" \
    s3://1000genomes/release/20130502/ \
    data/1000genomes/

/bin/bash: aws: command not found


Unlike the more modern high-coverage version of the thousand genomes (1KG) data, which provides the raw single-sample gVCF files TileDB-VCF was designed for, the low-coverage Phase 3 data provides only cohort-level project VCF (pVCF) files for each contig. Therefore, we must first split the pVCF file back into single-sample VCF files prior to ingestion.

In [ ]:
!bcftools +split \
    -Ob \
    -o data/split-bcfs \
    data/1000genomes/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz

Next we’ll filter the split VCF files to include only records with a non-reference allele and remove `INFO` attributes that are either static (e.g., `NS`) or cohort-specific and recoverable (e.g., `AF`). We’ll save the final pre-processed files in *BCF* format, which is a binary representation of the VCF format. The resulting filtered BCF files are a close approximation of the raw single sample VCF files typically stored for large population genomics projects.

In [ ]:
!rm_tags=INFO/AF,INFO/NS,INFO/EAS_AF,INFO/AMR_AF,INFO/AFR_AF,INFO/EUR_AF,INFO/SAS_AF

!ls data/split-bcfs/*.bcf | parallel -j16 \
    "bcftools view --min-ac 1 -Ou -s {/.} {} | bcftools annotate -Ob --remove $rm_tags -o data/filtered-bcfs/{/}"

The new BCF files must also be indexed:

In [ ]:
!ls data/filtered-bcfs/*.bcf | parallel -j16 "bcftools index {}"

For convenience, we’ll store these pre-processed files on S3:

In [ ]:
!aws s3 cp --recursive data/filtered-bcfs/ s3://genomic-datasets/notebooks/1kgp3/filtered-bcfs

## Storing Data in TileDB VCF

Next we will create a TileDB VCF dataset and ingest the VCF data.

The following was run on a `m5.4xlarge` system with a 300GB EBS volume to handle the large number of VCF files.

### Create the dataset

You can create a TileDB VCF dataset anywhere that TileDB supports, this could be a local filesystem, s3, azure, gcs, hdfs, and more. For this example we’ll create it on s3, the most common use case.

In [ ]:
# replace the path below with your own s3 bucket
!tiledbvcf create -u s3://genomic-datasets/notebooks/1kgp3/1kgp3-array

#### Storing the data

To store the VCF data in TileDB VCF you simply need a list of the VCF/BCF file locations (e.g., local file paths, S3 URIs, *etc*).

Run the following command to ingest the pre-processed BCF files into the TileDB-VCF dataset.

In [ ]:
!tiledbvcf store -u s3://genomic-datasets/notebooks/1kgp3/1kgp3-array \
    --threads 16 \
    --sample-batch-size 20 \
    --verbose \
    --stats \
    data/filtered-bcfs/*.bcf

When running with `--verbose` you will get a summary printed at the end: `Done. Ingested 857,389,804 records (+ 4,076,523 anchors) from 2,504 samples in 2,325.25 seconds.`

This indicates we’ve ingested over 857 million genomic positions into the TileDB VCF dataset. With the `m5.4xlarge` instance costing \$0.768 an hour, and this ingestion took just under 40 minutes, the cost of ingestion was \$0.50 USD.

The final array is 7.7Gb, or just under half the size of the individual compressed BCF files.

Following ingestion, it may help performance to consolidate the metadata fragments:

In [ ]:
$tiledbvcf utils consolidate fragment_meta -u s3://genomic-datasets/notebooks/1kgp3/1kgp3-array

## Reading, Analysis and Exporting

In this section we will walk through accessing the TileDB-VCF dataset with python and also exporting back to VCF and TSV.

### Python API

TileDB-VCF offers several APIs, for this section we will focus on the Python API. First we load the module and setup a few variables.

In [1]:
import tiledbvcf

uri = "s3://genomic-datasets/notebooks/1kgp3/1kgp3-array"
bedfile = "s3://genomic-datasets/notebooks/1kgp3/hg37_chr1_covidHgiGwasR4PvalC2_plog3.bed.gz"

#### Reading into Pandas Dataframe

Pandas is one of the most popular data science tools in python. TileDB VCF’s python API produces results directly into a pandas dataframe. This makes its easy to analyze the data and leverage any of pandas’ builtin algorithms.

Let’s run a typical query on the 1kg TileDB-VCF dataset we created above. We’ll retrieve all variants that overlap the gene *MTOR* on chr 1 for sample HG00096, along with a few attributes.

In [ ]:
ds = tiledbvcf.Dataset(uri, stats = True)

ds.read(
    attrs = ["sample_name", "contig", "pos_start", "pos_end", "alleles", "fmt_GT"], 
    regions = ["1:43337848-43352772"],
    samples = ["HG00096"]
)

If the `Dataset` object was created with `stats = True` you can print out a variety of useful information about the query:

In [ ]:
print(ds.tiledb_stats())

The above query took only 1.21914 secs for a single sample. Running this same query across all 2,504 samples took only 15.1911 secs.

For production-sized queries that encompass large portions of the genome it's more convenient to provide bed files with the query regions. Here, we'll use a bed file on s3 that contains 1,040 regions on chr1 that show at least a moderate association with with SARS-CoV-2 infection susceptibility (data obtained from the [COVID-19 Host Genetics Initiative](https://www.covid19hg.org/)).

In [ ]:
ds = tiledbvcf.Dataset(uri, verbose = True)

df = ds.read(
    attrs = ["sample_name", "contig", "pos_start", "pos_end", "alleles", "info_DP", "fmt_GT"], 
    samples = ds.samples()[:10],
    bed_file = bedfile
)

df

This query completed in 2.5 secs for 10 samples.

#### Filter Example

Using the pandas dataframe returned from TileDB-VCF we can apply additional filters. For instance if we wanted to filter the above result on read depth (`info_DP`):

In [ ]:
df[df.info_DP.apply(lambda x: x[0] > 5000)]

The python API supports a variety of advanced uses, batching, partitioning, dask and more. We are happy to follow-up with additional details beyond these initial examples.

### CLI Exporting

In addition to the python API it is also possible to export the dataset back into VCF format. This can be helpful in interoperating with legacy tools.

#### Exporting to VCF

When exporting to VCF you can specify any number of samples, and each will be exported to its own file in vcf, compressed vcf or bcf format depdning on what you set for `--output-format`.

For example to export the entire sample for `HG00096`, `HG00097`, and `HG00099` you can run the following:

In [ ]:
!tiledbvcf export --uri s3://genomic-datasets/notebooks/1kgp3/1kgp3-array \
    --output-format v \
    --sample-names HG00096,HG00097,HG00099 \
    --verbose

This produces 3 files: `HG00096.vcf`, `HG00097.vcf`, and `HG00099.vcf`.

##### Filtering Exports

You can also combine the use of regions (bed file or list of regions passed to cli) to filter the export.

In [ ]:
!tiledbvcf export --uri s3://genomic-datasets/notebooks/1kgp3/1kgp3-array \
    --output-format v \
    --sample-names HG00096,HG00097,HG00099 \
    --regions-file s3://genomic-datasets/notebooks/1kgp3/hg37_chr1_covidHgiGwasR4PvalC2_plog3.bed.gz \
    --verbose

This will also produce the 3 VCF files, like the previous export. However, these files are filtered for the same SARS-CoV-2 associated genomic regions specified in the bed file.


#### Exporting to TSV

For even more generic usecases you can export data to tab seperate files with the `--output-format t` option.

In [ ]:
$tiledbvcf export --uri s3://genomic-datasets/notebooks/1kgp3/1kgp3-array \
    --output-format t \
    --tsv-fields CHR,POS,I:END,REF,ALT,S:GT,Q:POS,Q:END \
    --sample-names HG00096,HG00097,HG00099 \
    --regions-file s3://genomic-datasets/notebooks/1kgp3/hg37_chr1_covidHgiGwasR4PvalC2_plog3.bed.gz \
    --verbose --output-path sars-cov-2-associated-regions.tsv